In [1]:
import requests
from bs4 import BeautifulSoup
import pickle
import re
import numpy as np
from tqdm import tqdm
import idna
from fake_useragent import UserAgent
import urllib3
import pandas as pd
import concurrent.futures
import os
import codecs


# Disable warnings issued by urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def cleanTxt(text):
    '''Function is called for cleaning text from trash
    INPUT: dirty string
    OUTPUT: More or less clean string'''
    
    
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

    text = re.sub(r'@[А-Яа-я0-9]+', '', text)  # Remove @
    text = re.sub(r'#', '', text)  # remove #
    text = re.sub(r'{', '', text)
    text = re.sub(r'}', '', text)
#     text = re.sub(r'.', ' ', text)
#     text = re.sub(r',', ' ', text)
#     text = re.sub(r'/', '', text)
#     text = re.sub(r'\\', '', text)
    text = re.sub('^а-яА-Я', ' ', text)
    text = re.sub(r'https?:\/\/\S+', '', text)  # remove hyperlink
    text = re.sub(r'\n', '', text)
    text = re.sub(emoj, '', text)
    
    
    

    text = text.strip()
    text = text.lower()
    #text = text.split()
    return text

In [3]:
with open('E:/mts/unique_values_dict.pkl', 'rb') as f:
    urls = pickle.load(f)
urls = list(urls['url_host'].keys())

In [4]:
result = {}
errors = {}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
#     'Referer': 'https://www.google.com/',
#     'Connection': 'keep-alive'
}

def process_url(url):
    try:
        url = idna.decode(url)
    except:
        url = idna.decode(url.encode('ascii'))

    if '.turbopages.org' in url:
        url = url.replace('.turbopages.org', '').replace('--', '_').replace('-', '.')

    try:
        response = requests.get('http://' + url, headers=headers, verify=False, timeout=10)      
        
        if response.status_code == 200:
            
            if response.encoding == 'ISO-8859-1':
                
                response.encoding = 'UTF-8'
                soup = BeautifulSoup(response.text, 'html.parser')
            
    #             if soup.find('meta', )
                title = soup.title.text.strip()

                try:
                    header = soup.find('h1').text.strip()
                except:
                    header = ''
                try:
                    description = soup.find('meta', attrs={'name': 'description'})['content'].strip()
                except:
                    description = ''

                try:
                    keywords = soup.find('meta', attrs={'name': 'keywords'})['content'].strip()
                except:
                    keywords = '' 
                    
                response.encoding = 'windows-1251'
                soup = BeautifulSoup(response.text, 'html.parser')

                title_ = soup.title.text.strip()

                try:
                    header_ = soup.find('h1').text.strip()
                except:
                    header_ = ''
                try:
                    description_ = soup.find('meta', attrs={'name': 'description'})['content'].strip()
                except:
                    description_ = ''

                try:
                    keywords_ = soup.find('meta', attrs={'name': 'keywords'})['content'].strip()
                except:
                    keywords_ = '' 
                    
                data = str(title) + ' ' + str(header) + ' ' + str(description) + ' ' + str(keywords) +\
                            str(title_) + ' ' + str(header_) + ' ' + str(description_) + ' ' + str(keywords)
                    
            else:
                soup = BeautifulSoup(response.text, 'html.parser')
            
                title = soup.title.text.strip()

                try:
                    header = soup.find('h1').text.strip()
                except:
                    header = ''
                try:
                    description = soup.find('meta', attrs={'name': 'description'})['content'].strip()
                except:
                    description = ''

                try:
                    keywords = soup.find('meta', attrs={'name': 'keywords'})['content'].strip()
                except:
                    keywords = '' 
                
                data = str(title) + ' ' + str(header) + ' ' + str(description) + ' ' + str(keywords)
                
            data = cleanTxt(data)
            result[url] = data
        else:
#             print(f"Error fetching URL {url}. Status code: {response.status_code}")
            result[url] = np.NaN
            if response.status_code not in errors:
                errors[response.status_code] = []
            errors[response.status_code].append(url)

    except requests.exceptions.RequestException as e:
#         print(f'Error fetching URL {url}: {e}')
        result[url] = np.NaN
#         if e not in  errors:
#             errors[e] = []
#         errors[e].append(url)

In [5]:

with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    executor.map(process_url, urls)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the Beautiful

In [6]:
result = pd.DataFrame(data={'url_host': result.keys(), 'url_info': result.values()})
result.to_csv('E:/mts/scrap/urls_info_0.csv', index=False)

In [7]:
errors = pd.DataFrame(data={'error': errors.keys(), 'urls': errors.values()})
errors.to_csv('E:/mts/scrap/errors_4_0.csv', index=False)

In [12]:
a = pd.read_csv('E:/mts/scrap/urls_info_0.csv')

In [25]:
a

,url_host,url_info
0,last-frontier.ru,главная страница - ильинский рубеж фильм про ...
1,app-143608.games.s3.yandex.net,NaN
2,обувьлига.рф,обувной центр лига. обувной центр лига обувной...
3,manasplav.ru,сплав по мане -лето 2023 года! сплав по мане -...
4,przspb.ru,www.przspb.ru | — наша цель — успех! — наша це...
...,...,...
189158,best-horse.com,best-horse все для всадника и лошади спорта be...
189159,sch24.rybadm.ru,NaN
189160,ivanovo.orgsprav.com,справочник города иваново. ивановская область ...
189161,biysk.promportal.su,"promportal.su - каталог товаров и услуг, интер..."


# не забыть почистить адреса в главном датасете

# в назваиях некоторых сайтов есть имена
